# Collecting inel datasets locally

In [1]:
evn_path='/Users/yuriypeshkichev/Projects/huawei-nlp/final_project/corpora/evenki-1.0-noaudio/'

xas_path='/Users/yuriypeshkichev/Projects/huawei-nlp/final_project/corpora/kamas-1.0-noaudio/'

out_path='/Users/yuriypeshkichev/Projects/huawei-nlp/final_project/'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import re
import pandas as pd
import xml.etree.ElementTree as ET

In [4]:
from cyrillicToLatin import cyrToIPA

In [23]:
def clean_source(string):
    a=re.compile("<.*?>")
    txt=a.sub('',string)
    txt = re.sub("[\(\[].*?[\)\]]", "", txt).lower()
    txt = re.sub(r'{[^}]*}','',txt)
    spl = txt.split('/')
    if len(spl) > 2:
        txt = ''.join((spl[0], spl[-1]))
    txt = txt.replace('/','')
     
    txt = txt.replace('*', '')
    txt = txt.replace('"', '')
    txt = txt.replace('«', '')
    txt = txt.replace('»', '')
    txt = txt.replace('ʼ', '')
    txt = txt.replace('—','')
    txt = txt.replace('ˊ','')
    txt = txt.replace('ˋ','')
    
    txt = " ".join(txt.split())
    
    
    # txt = txt.replace('с', 's')
    # txt = txt.replace('a', 'а')
    # txt = txt.replace('л', 'l')
    # txt = txt.replace('и', 'i')
    # txt = txt.replace('р', 'p')
    # txt = txt.replace('п', 'n')    
    # txt = txt.replace('к', 'k')    
    # txt = txt.replace('е', 'e')
    # txt = txt.replace('й','j')    
    # txt = txt.replace('o','о')
    # txt = txt.replace('х','x')
    # txt = txt.replace('т','t')
    # txt = txt.replace('в','v')
                      
    txt = txt.replace("'",'')
    
    txt = re.sub(r'(?<=[.,])(?=[^\s])', r' ', txt)
    txt = re.sub(r'[.,"\'?:!;]', '', txt)
    txt = txt.replace("[",'')
    txt = txt.replace("]",'')
    txt = txt.replace("(",'')
    txt = txt.replace(")",'')
    
#     txt = txt.replace('ˊ','')
#     txt = txt.replace('ˋ','')
#     txt = txt.replace('ң','ӈ')



#     txt = txt.replace('w','в')
#     txt = txt.replace('џ','')
#     txt = txt.replace('ӱ','у')
#     txt = txt.replace('ӓ','а̄')
#     txt = txt.replace('і','и')
#     txt = txt.replace('ӧ','о̄')
#     txt = txt.replace('ѣ','е')
#     txt = txt.replace('йа','я')
    
#     txt = txt.replace('ӯ','у')
#     txt = txt.replace('ы̄','ы')


#     txt = txt.replace('c','с')

#     txt = txt.replace('y','у')
#     txt = txt.replace('u','и')
#     txt = txt.replace('o','о')
#     txt = txt.replace('p','р')



    
    return txt

In [24]:
def clean_target(string):
    a=re.compile("<.*?>")
    txt=a.sub('',string)
    txt = re.sub("[\(\[].*?[\)\]]", "", txt).lower()
    txt = re.sub(r'{[^}]*}','',txt)
    
    spl = txt.split('/')
    if len(spl) > 2:
        txt = ''.join((spl[0], spl[-1]))
    txt = txt.replace('/','')
    
    
    txt = txt.replace('*', '')
    txt = txt.replace('"', '')
    txt = txt.replace('«', '')
    txt = txt.replace('»', '')
    txt = txt.replace('ʼ', '')
    txt = txt.replace('—','')
    txt = " ".join(txt.split())

    
    txt = txt.replace("'",'')
    txt = txt.replace("/",'')
    
    txt = re.sub(r'(?<=[.,])(?=[^\s])', r' ', txt)
    txt = re.sub(r'[.,"\'?:!;]', '', txt)
    txt = txt.replace("[",'')
    txt = txt.replace("]",'')
    txt = txt.replace("(",'')
    txt = txt.replace(")",'')
    
    txt = txt.replace('a', 'а')
    txt = txt.replace('o','о')
    txt = txt.replace('k', 'к')
    
    return txt

In [ ]:
for path in Path(data_path).rglob('*.exb'):
    tree = ET.parse(path)
    root = tree.getroot()
    for tier in root.findall('basic-body/tier'):
        if tier.attrib['id'] == 'ts' or tier.attrib['id'] == 'fr':
            print("tier:", tier.attrib['id'])
            for child in tier:
                print (child.text)
                #print (child.attrib['start'])

In [67]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [25]:
#XAS
dict1 = {}
for path in Path(xas_path).rglob('*.exb'):
    tree = ET.parse(path)
    root = tree.getroot()
    src_list = []
    rus_list = []
    src_time = []
    rus_time = []
    for tier in root.findall('basic-body/tier'):
        if "ts" in tier.attrib['id']:
            for child in tier:
                if (not child.text) or (child.text == '1'):
                    rus_list = []
                    src_list = []
                    src_time = []
                    rus_time = []
                    break
                txt = clean_source(child.text)
                txt = cyrToIPA(txt)
                if txt:
                    src_list.append(txt)
                    src_time.append(child.attrib['start'])
        if 'fr' in tier.attrib['id']:
            for child in tier:
                if (not child.text) or (child.text == '1'):
                    rus_list = []
                    src_list = []
                    src_time = []
                    rus_time = []
                    break
                #print(child.text)
                txt = clean_target(child.text)
                if txt:
                    rus_list.append(txt)
                    rus_time.append(child.attrib['start'])

        if src_list and rus_list:
            sd = {"time":src_time,"text":src_list}
            rd = {"time":rus_time,"text":rus_list}
            sf = pd.DataFrame(data=sd)
            rf = pd.DataFrame(data=rd)
            #print(sf['time'][sf['time'].isin(rf['time'])])
            #print(sf['time'])
            #print("r:",rf['time'])
            new_sf = sf[sf['time'].isin(rf['time'])]
            new_rf = rf[rf['time'].isin(sf['time'])]
            #print(new_rf)
            # if len(evn_list) != len(rus_list):
            #     rus_list = []
            #     evn_list = []
            #     break
            #print(evn_list)
            if 'src' in dict1.keys():
                #print(dict['src'])
                dict1['src'] = pd.concat([dict1['src'],new_sf['text']])
                #print(len(dict1['src']))
            else:
                dict1['src'] = new_sf['text']
            if 'rus' in dict1.keys():
                dict1['rus'] = pd.concat([dict1['rus'],new_rf['text']])
            else:
                dict1['rus'] = new_rf['text']
            rus_list = []
            src_list = []
            src_time = []
            rus_time = []

In [26]:
#EVN
dict2 = {}
for path in Path(evn_path).rglob('*.exb'):
    tree = ET.parse(path)
    root = tree.getroot()
    src_list = []
    rus_list = []
    src_time = []
    rus_time = []
    for tier in root.findall('basic-body/tier'):
        #print(tier.attrib['id'])
        if "stl" in tier.attrib['id']:
            for child in tier:
                if (not child.text) or (child.text == '1'):
                    rus_list = []
                    src_list = []
                    src_time = []
                    rus_time = []
                    break
                txt = clean_source(child.text)
                txt = cyrToIPA(txt)
                
                if txt:
                    src_list.append(txt)
                    src_time.append(child.attrib['start'])
        if "fr" in tier.attrib['id']:
            for child in tier:
                if (not child.text) or (child.text == '1'):
                    rus_list = []
                    src_list = []
                    src_time = []
                    rus_time = []
                    break
                #print(child.text)
                txt = clean_target(child.text)
                if txt:
                    rus_list.append(txt)
                    rus_time.append(child.attrib['start'])

        if src_list and rus_list:
            sd = {"time":src_time,"text":src_list}
            rd = {"time":rus_time,"text":rus_list}
            sf = pd.DataFrame(data=sd)
            rf = pd.DataFrame(data=rd)
            #print(sf['time'][sf['time'].isin(rf['time'])])
            #print(sf['time'])
            #print("r:",rf['time'])
            new_sf = sf[sf['time'].isin(rf['time'])]
            new_rf = rf[rf['time'].isin(sf['time'])]
            # new_sf = sf
            # new_rf = rf
            #print(new_rf)
            # if len(evn_list) != len(rus_list):
            #     rus_list = []
            #     evn_list = []
            #     break
            #print(evn_list)
            if 'src' in dict2.keys():
                #print(dict2['src'])
                dict2['src'] = pd.concat([dict2['src'],new_sf['text']])
                #print(len(dict2['src']))
            else:
                dict2['src'] = new_sf['text']
            if 'rus' in dict2.keys():
                dict2['rus'] = pd.concat([dict2['rus'],new_rf['text']])
            else:
                dict2['rus'] = new_rf['text']
            rus_list = []
            src_list = []
            src_time = []
            rus_time = []

In [28]:
with open(out_path+"xas-inel.tsv","w") as fw:
    for a,b in zip(dict1['src'],dict1['rus']):
        fw.write(a+"\t"+b+"\n")

In [29]:
with open(out_path+"evn-inel.tsv","w") as fw:
    for a,b in zip(dict2['src'],dict2['rus']):
        fw.write(a+"\t"+b+"\n")